In [1]:
import torch
import esm

In [ ]:
sequences = ["MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNG<mask><mask><mask>EWTYDDATKTFT<mask>TE"]
variants = ["<mask><mask><mask><mask>"]

In [2]:
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()

In [3]:
alphabet.mask_idx

32

In [54]:
from torch.utils.data import Dataset

class SequenceDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        src = self.sequences[idx]
        return src

In [71]:
from torch.utils.data import DataLoader

sequences = torch.tensor([[6, 7, 8, 9, 10, 11], [6, 7, 8, 9, 10, 11], [6, 7, 8, 9, 10, 11]])
dataset = SequenceDataset(sequences)
dataloader = DataLoader(dataset, batch_size=2)

In [85]:
model.train()

loss_function = torch.nn.CrossEntropyLoss(reduction='mean')

batches = [b for b in dataloader]
batch = batches[0]
batch

input = batch.clone()
input[:, 3] = alphabet.mask_idx
logits = model(input)['logits']
logits = logits.view(-1, len(alphabet.all_toks))
loss = loss_function(logits, batch.view(-1))
loss.backward()

In [82]:
logits

tensor([[ 17.2130, -10.4268,   4.3073, -10.3417,  -5.0231,  -5.5713,  -5.3020,
          -6.2960,  -6.6767,  -7.1270,  -7.2446,  -6.9955,  -4.9150,  -4.8448,
          -4.8673,  -7.4093,  -7.5647,  -6.7933,  -4.7558,  -6.4050,  -4.1163,
          -6.4284,  -6.9402,  -5.1816, -16.4025,  -8.9111,  -5.1899,  -6.8634,
         -15.3043, -15.1924, -14.0209, -15.4579, -10.3194],
        [-14.2885, -17.6840, -10.2384, -17.5906,  -0.5864,  -0.6038,  -1.0612,
           2.2265,  -0.4483,  -1.4295,  -0.6930,  -1.0180,  -1.5467,  -1.6886,
          -0.9941,  -1.7375,  -1.8032,  -2.0228,  -1.7846,  -2.1204,   1.0479,
          -2.2927,  -2.4638,  -1.7659,  -2.8254,  -9.7229, -11.3728, -11.1799,
         -16.4022, -16.2923, -16.1494, -16.4295, -17.6588],
        [-14.8074, -17.2731,  -9.4569, -17.2336,  -0.3289,  -0.1936,  -0.4727,
          -0.5211,   3.0008,  -0.8466,  -0.1246,  -0.6098,  -0.9388,  -1.2826,
          -0.6558,  -0.8636,  -1.1136,  -1.2914,  -1.3697,  -1.6351,  -1.9211,
          -

In [79]:
loss

tensor(8.7067, grad_fn=<NllLossBackward0>)

In [11]:
model.eval()
out = model(sequences)

In [17]:
out['logits'].size()

torch.Size([3, 6, 33])

In [23]:
logits = out['logits']
logits = logits.view(-1, len(alphabet.all_toks))
logits.size()

torch.Size([18, 33])

In [53]:
loss_function = torch.nn.CrossEntropyLoss(reduction='mean')
loss_function(logits, sequences.view(-1))
#loss(torch.logit(torch.tensor([[0.98, 0.01, 0.01], [0.01, 0.98, 0.01]])), torch.tensor([0, 1]))

tensor(11.5965, grad_fn=<NllLossBackward0>)

In [25]:
a = torch.tensor([[[0, 0, 0, 0], [1, 1, 1, 1], [0, 0, 0, 0]], [[0, 0, 3, 0], [1, 1, 1, 1], [0, 0, 0, 0]]])
a = a.view(-1, 4)
a

tensor([[0, 0, 0, 0],
        [1, 1, 1, 1],
        [0, 0, 0, 0],
        [0, 0, 3, 0],
        [1, 1, 1, 1],
        [0, 0, 0, 0]])